In [3]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [4]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [5]:


gamma=0.9
epsilon_decay = 0.005
learning_rate =0.0005
max_e=1
min_e=0
replay_batch_size=30
iterations=2000

In [6]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
target_model =initiate_NN(learning_rate)
target_model.set_weights(online_model.get_weights())
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=3000:
        memory = memory[-3000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -370.98  rolling reward:  -370.98  Steps:  94  epsilon:  1.00
episode:  1  episode reward:  -109.99  rolling reward:  -240.48  Steps:  121  epsilon:  1.00
episode:  2  episode reward:  -139.88  rolling reward:  -206.95  Steps:  95  epsilon:  0.99
episode:  3  episode reward:  -272.16  rolling reward:  -223.25  Steps:  116  epsilon:  0.99
episode:  4  episode reward:  -94.84  rolling reward:  -197.57  Steps:  66  epsilon:  0.98
episode:  5  episode reward:  -124.43  rolling reward:  -185.38  Steps:  72  epsilon:  0.98
episode:  6  episode reward:  -95.06  rolling reward:  -172.48  Steps:  60  epsilon:  0.97
episode:  7  episode reward:  -155.70  rolling reward:  -170.38  Steps:  97  epsilon:  0.97
episode:  8  episode reward:  12.90  rolling reward:  -150.02  Steps:  94  epsilon:  0.96
episode:  9  episode reward:  1.75  rolling reward:  -134.84  Steps:  114  epsilon:  0.96
episode:  10  episode reward:  -64.62  rolling reward:  -128.45  Steps:  79  epsilon

episode:  89  episode reward:  -134.83  rolling reward:  -139.84  Steps:  132  epsilon:  0.64
episode:  90  episode reward:  -111.94  rolling reward:  -139.53  Steps:  88  epsilon:  0.64
episode:  91  episode reward:  -83.34  rolling reward:  -138.92  Steps:  116  epsilon:  0.63
episode:  92  episode reward:  -114.24  rolling reward:  -138.66  Steps:  156  epsilon:  0.63
episode:  93  episode reward:  -134.68  rolling reward:  -138.61  Steps:  188  epsilon:  0.63
episode:  94  episode reward:  -92.34  rolling reward:  -138.13  Steps:  105  epsilon:  0.63
episode:  95  episode reward:  -149.26  rolling reward:  -138.24  Steps:  143  epsilon:  0.62
episode:  96  episode reward:  -86.74  rolling reward:  -137.71  Steps:  91  epsilon:  0.62
episode:  97  episode reward:  -121.30  rolling reward:  -137.54  Steps:  129  epsilon:  0.62
episode:  98  episode reward:  -40.60  rolling reward:  -136.56  Steps:  142  epsilon:  0.61
episode:  99  episode reward:  -126.25  rolling reward:  -136.46  

episode:  177  episode reward:  -225.91  rolling reward:  -85.13  Steps:  265  epsilon:  0.41
episode:  178  episode reward:  -72.23  rolling reward:  -85.26  Steps:  198  epsilon:  0.41
episode:  179  episode reward:  -41.42  rolling reward:  -84.65  Steps:  86  epsilon:  0.41
episode:  180  episode reward:  -93.12  rolling reward:  -84.09  Steps:  247  epsilon:  0.41
episode:  181  episode reward:  3.58  rolling reward:  -82.48  Steps:  1000  epsilon:  0.40
episode:  182  episode reward:  -286.55  rolling reward:  -83.85  Steps:  239  epsilon:  0.40
episode:  183  episode reward:  -118.60  rolling reward:  -83.85  Steps:  113  epsilon:  0.40
episode:  184  episode reward:  -87.70  rolling reward:  -83.28  Steps:  374  epsilon:  0.40
episode:  185  episode reward:  -268.18  rolling reward:  -84.71  Steps:  459  epsilon:  0.40
episode:  186  episode reward:  -247.43  rolling reward:  -86.63  Steps:  327  epsilon:  0.39
episode:  187  episode reward:  -25.48  rolling reward:  -85.81  St

KeyboardInterrupt: 

In [8]:

for episode in range(195,iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=3000:
        memory = memory[-3000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  195  episode reward:  -92.60  rolling reward:  -81.40  Steps:  269  epsilon:  0.38
episode:  196  episode reward:  -25.93  rolling reward:  -79.89  Steps:  242  epsilon:  0.38
episode:  197  episode reward:  113.46  rolling reward:  -78.11  Steps:  1000  epsilon:  0.37
episode:  198  episode reward:  11.41  rolling reward:  -77.45  Steps:  123  epsilon:  0.37
episode:  199  episode reward:  -87.23  rolling reward:  -78.31  Steps:  626  epsilon:  0.37
episode:  200  episode reward:  6.75  rolling reward:  -77.36  Steps:  110  epsilon:  0.37
episode:  201  episode reward:  -39.54  rolling reward:  -74.49  Steps:  222  epsilon:  0.37
episode:  202  episode reward:  -226.99  rolling reward:  -75.67  Steps:  194  epsilon:  0.36
episode:  203  episode reward:  -13.31  rolling reward:  -74.50  Steps:  104  epsilon:  0.36
episode:  204  episode reward:  37.54  rolling reward:  -73.09  Steps:  1000  epsilon:  0.36
episode:  205  episode reward:  -59.85  rolling reward:  -73.00  Steps:

episode:  284  episode reward:  70.73  rolling reward:  -27.52  Steps:  1000  epsilon:  0.24
episode:  285  episode reward:  44.11  rolling reward:  -27.42  Steps:  247  epsilon:  0.24
episode:  286  episode reward:  199.27  rolling reward:  -23.41  Steps:  702  epsilon:  0.24
episode:  287  episode reward:  -7.85  rolling reward:  -23.89  Steps:  127  epsilon:  0.24
episode:  288  episode reward:  83.60  rolling reward:  -22.78  Steps:  1000  epsilon:  0.24
episode:  289  episode reward:  163.04  rolling reward:  -22.08  Steps:  1000  epsilon:  0.24
episode:  290  episode reward:  79.46  rolling reward:  -21.02  Steps:  1000  epsilon:  0.23
episode:  291  episode reward:  -111.57  rolling reward:  -19.61  Steps:  104  epsilon:  0.23
episode:  292  episode reward:  -22.75  rolling reward:  -17.96  Steps:  270  epsilon:  0.23
episode:  293  episode reward:  -57.53  rolling reward:  -18.20  Steps:  110  epsilon:  0.23
episode:  294  episode reward:  210.85  rolling reward:  -14.82  Steps

episode:  372  episode reward:  -92.26  rolling reward:  -32.43  Steps:  1000  epsilon:  0.16
episode:  373  episode reward:  -84.29  rolling reward:  -31.70  Steps:  1000  epsilon:  0.15
episode:  374  episode reward:  -87.26  rolling reward:  -33.16  Steps:  1000  epsilon:  0.15
episode:  375  episode reward:  -71.30  rolling reward:  -33.98  Steps:  1000  epsilon:  0.15
episode:  376  episode reward:  -92.14  rolling reward:  -33.90  Steps:  1000  epsilon:  0.15
episode:  377  episode reward:  -48.83  rolling reward:  -33.66  Steps:  1000  epsilon:  0.15
episode:  378  episode reward:  -70.45  rolling reward:  -34.86  Steps:  353  epsilon:  0.15
episode:  379  episode reward:  -0.52  rolling reward:  -34.97  Steps:  255  epsilon:  0.15
episode:  380  episode reward:  -51.94  rolling reward:  -36.47  Steps:  1000  epsilon:  0.15
episode:  381  episode reward:  208.99  rolling reward:  -34.83  Steps:  717  epsilon:  0.15
episode:  382  episode reward:  244.06  rolling reward:  -32.96 

episode:  460  episode reward:  -137.01  rolling reward:  -67.17  Steps:  1000  epsilon:  0.10
episode:  461  episode reward:  -87.64  rolling reward:  -67.82  Steps:  1000  epsilon:  0.10
episode:  462  episode reward:  -108.64  rolling reward:  -70.35  Steps:  1000  epsilon:  0.10
episode:  463  episode reward:  -73.14  rolling reward:  -70.56  Steps:  1000  epsilon:  0.10
episode:  464  episode reward:  -98.57  rolling reward:  -71.30  Steps:  1000  epsilon:  0.10
episode:  465  episode reward:  -70.44  rolling reward:  -74.43  Steps:  1000  epsilon:  0.10
episode:  466  episode reward:  -77.72  rolling reward:  -74.98  Steps:  1000  epsilon:  0.10
episode:  467  episode reward:  -67.49  rolling reward:  -74.48  Steps:  1000  epsilon:  0.10
episode:  468  episode reward:  -141.71  rolling reward:  -75.26  Steps:  1000  epsilon:  0.10
episode:  469  episode reward:  -77.22  rolling reward:  -74.81  Steps:  1000  epsilon:  0.10
episode:  470  episode reward:  -80.83  rolling reward:  

episode:  548  episode reward:  -100.87  rolling reward:  -86.41  Steps:  1000  epsilon:  0.06
episode:  549  episode reward:  -116.28  rolling reward:  -86.73  Steps:  1000  epsilon:  0.06
episode:  550  episode reward:  -138.77  rolling reward:  -87.74  Steps:  1000  epsilon:  0.06
episode:  551  episode reward:  -107.26  rolling reward:  -87.74  Steps:  1000  epsilon:  0.06
episode:  552  episode reward:  -114.13  rolling reward:  -88.59  Steps:  1000  epsilon:  0.06
episode:  553  episode reward:  -110.63  rolling reward:  -88.70  Steps:  1000  epsilon:  0.06
episode:  554  episode reward:  -91.08  rolling reward:  -88.50  Steps:  1000  epsilon:  0.06
episode:  555  episode reward:  -13.19  rolling reward:  -87.80  Steps:  1000  epsilon:  0.06
episode:  556  episode reward:  -68.97  rolling reward:  -86.95  Steps:  1000  epsilon:  0.06
episode:  557  episode reward:  -142.70  rolling reward:  -87.13  Steps:  1000  epsilon:  0.06
episode:  558  episode reward:  -76.79  rolling rewar

...

episode:  722  episode reward:  -73.98  rolling reward:  -97.59  Steps:  1000  epsilon:  0.03
episode:  723  episode reward:  -108.39  rolling reward:  -97.99  Steps:  1000  epsilon:  0.03
episode:  724  episode reward:  -80.96  rolling reward:  -97.32  Steps:  1000  epsilon:  0.03
episode:  725  episode reward:  -87.87  rolling reward:  -98.06  Steps:  1000  epsilon:  0.03
episode:  726  episode reward:  -77.46  rolling reward:  -98.49  Steps:  1000  epsilon:  0.03
episode:  727  episode reward:  -100.07  rolling reward:  -98.51  Steps:  1000  epsilon:  0.03
episode:  728  episode reward:  -99.54  rolling reward:  -97.74  Steps:  1000  epsilon:  0.03
episode:  729  episode reward:  -132.38  rolling reward:  -98.28  Steps:  1000  epsilon:  0.03
episode:  730  episode reward:  -113.88  rolling reward:  -98.59  Steps:  1000  epsilon:  0.03
episode:  731  episode reward:  -24.11  rolling reward:  -98.81  Steps:  1000  epsilon:  0.03
episode:  732  episode reward:  -107.64  rolling reward:

episode:  809  episode reward:  -106.75  rolling reward:  -90.08  Steps:  1000  epsilon:  0.02
episode:  810  episode reward:  -73.74  rolling reward:  -89.66  Steps:  1000  epsilon:  0.02
episode:  811  episode reward:  -29.28  rolling reward:  -89.19  Steps:  1000  epsilon:  0.02
episode:  812  episode reward:  -98.30  rolling reward:  -89.21  Steps:  1000  epsilon:  0.02
episode:  813  episode reward:  -118.83  rolling reward:  -89.53  Steps:  1000  epsilon:  0.02
episode:  814  episode reward:  -148.52  rolling reward:  -89.60  Steps:  1000  epsilon:  0.02
episode:  815  episode reward:  -103.38  rolling reward:  -89.36  Steps:  1000  epsilon:  0.02
episode:  816  episode reward:  -118.52  rolling reward:  -89.16  Steps:  1000  epsilon:  0.02
episode:  817  episode reward:  -81.68  rolling reward:  -89.02  Steps:  1000  epsilon:  0.02
episode:  818  episode reward:  -78.12  rolling reward:  -88.84  Steps:  1000  epsilon:  0.02
episode:  819  episode reward:  -84.99  rolling reward:

episode:  896  episode reward:  -62.37  rolling reward:  -89.37  Steps:  1000  epsilon:  0.01
episode:  897  episode reward:  -63.37  rolling reward:  -89.16  Steps:  1000  epsilon:  0.01
episode:  898  episode reward:  -44.25  rolling reward:  -88.51  Steps:  1000  epsilon:  0.01
episode:  899  episode reward:  -57.87  rolling reward:  -87.79  Steps:  1000  epsilon:  0.01
episode:  900  episode reward:  -62.55  rolling reward:  -87.30  Steps:  1000  epsilon:  0.01
episode:  901  episode reward:  -88.29  rolling reward:  -87.18  Steps:  1000  epsilon:  0.01
episode:  902  episode reward:  -80.80  rolling reward:  -87.36  Steps:  1000  epsilon:  0.01
episode:  903  episode reward:  -66.53  rolling reward:  -86.90  Steps:  1000  epsilon:  0.01
episode:  904  episode reward:  -189.34  rolling reward:  -87.51  Steps:  754  epsilon:  0.01
episode:  905  episode reward:  -77.26  rolling reward:  -87.28  Steps:  1000  epsilon:  0.01
episode:  906  episode reward:  -42.44  rolling reward:  -86

episode:  983  episode reward:  -133.52  rolling reward:  -93.11  Steps:  1000  epsilon:  0.01
episode:  984  episode reward:  -70.53  rolling reward:  -92.75  Steps:  1000  epsilon:  0.01
episode:  985  episode reward:  -109.25  rolling reward:  -92.56  Steps:  1000  epsilon:  0.01
episode:  986  episode reward:  -60.56  rolling reward:  -92.71  Steps:  1000  epsilon:  0.01
episode:  987  episode reward:  -119.99  rolling reward:  -92.60  Steps:  1000  epsilon:  0.01
episode:  988  episode reward:  -120.54  rolling reward:  -94.55  Steps:  1000  epsilon:  0.01
episode:  989  episode reward:  -128.04  rolling reward:  -95.02  Steps:  1000  epsilon:  0.01
episode:  990  episode reward:  -16.39  rolling reward:  -93.92  Steps:  1000  epsilon:  0.01
episode:  991  episode reward:  -101.64  rolling reward:  -94.01  Steps:  1000  epsilon:  0.01
episode:  992  episode reward:  -96.19  rolling reward:  -93.91  Steps:  1000  epsilon:  0.01
episode:  993  episode reward:  -50.37  rolling reward

KeyboardInterrupt: 

In [9]:
online_model.save_weights('weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 